In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
filepath = '/kaggle/input/datathon2023'
df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test_x.csv'), index_col=[0])
sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'), index_col=[0])

In [3]:
# Veri setinden 18-30 yaş arasında olup emekli olan kişileri çıkardım
df_train = df_train[~((df_train['Yaş Grubu'] == '18-30') & (df_train['İstihdam Durumu'] == 'Emekli'))]
# toplam 26 kişi

In [4]:
# Veri setinden 60 yaş üstü olup eğitime devam eden kişileri çıkardım
df_train = df_train[~((df_train['Yaş Grubu'] == '>60') & (df_train['Eğitime Devam Etme Durumu'] == 'Ediyor'))]
# toplam 25 kişi

In [5]:
# Veri setinden eğitimsiz olup eğitime devam eden kişileri çıkardım
df_train = df_train[~((df_train['Eğitim Düzeyi'] == 'Eğitimsiz') & (df_train['Eğitime Devam Etme Durumu'] == 'Ediyor'))]
# toplam 23 kişi

In [6]:
# obek_2'deki erkekleri de veriden çıkarıyorum
df_train = df_train[~((df_train['Cinsiyet'] == 'Erkek') & (df_train['Öbek İsmi'] == 'obek_2'))]
# toplam 19 kişi

In [7]:
# obek_4'deki yaşlıları da veriden çıkarıyorum
df_train = df_train[~((df_train['Yaş Grubu'] == '>60') & (df_train['Öbek İsmi'] == 'obek_4'))]
df_train = df_train[~((df_train['Yaş Grubu'] == '51-60') & (df_train['Öbek İsmi'] == 'obek_4'))]
df_train = df_train[~((df_train['Yaş Grubu'] == '41-50') & (df_train['Öbek İsmi'] == 'obek_4'))]
# toplam 16 kişi

In [8]:
# obek_5'de kırsalda yaşayan adamı veriden çıkarıyorum
df_train = df_train[~((df_train['Yaşadığı Şehir'] == 'Kırsal') & (df_train['Öbek İsmi'] == 'obek_5'))]
# 1 kişi

In [9]:
# obek_6'da gençleri veriden çıkarıyorum ayrıca eğitime devam edenleri de çıkarıyorum çünkü burası yaşlı grubu
df_train = df_train[~((df_train['Yaş Grubu'] == '18-30') & (df_train['Öbek İsmi'] == 'obek_6'))]
df_train = df_train[~((df_train['Eğitime Devam Etme Durumu'] == 'Ediyor') & (df_train['Öbek İsmi'] == 'obek_6'))]
# 31 kişi

In [10]:
# obek_7'de işsiz olanları veriden çıkarıyorum. outliers hepsi. Ayrıca kırsalda yaşayanlar da öyle.
df_train = df_train[~((df_train['İstihdam Durumu'] == 'İşsiz veya Düzenli Bir İşi Yok') & (df_train['Öbek İsmi'] == 'obek_7'))]
df_train = df_train[~((df_train['Yaşadığı Şehir'] == 'Kırsal') & (df_train['Öbek İsmi'] == 'obek_7'))]
# 10 kişi

In [11]:
# obek_8'de genç olmayan herkesi çıkarıcaz.
df_train = df_train[~((df_train['Yaş Grubu'] != '18-30') & (df_train['Öbek İsmi'] == 'obek_8'))]
# 1 de kırsal vardı da tamam o da o yaş grubundaymış.
# toplam kişi 20

# Encode

In [12]:
# Encoding kısmına gelecek olursak, kategorik değişkenleri binary olanlar ve multiclass olanlar olarak ayırmalıyız.

# Öncelikle cinsiyet, medeni durum ve eğitime devam etme durumunu label encoding ile dönüştürelim.
df_train['Cinsiyet'] = df_train['Cinsiyet'].replace({'Kadın': 0, 'Erkek': 1})
df_test['Cinsiyet']  = df_test['Cinsiyet'].replace({'Kadın': 0, 'Erkek': 1})

df_train['Medeni Durum'] = df_train['Medeni Durum'].replace({'Bekar': 0, 'Evli': 1})
df_test['Medeni Durum']  = df_test['Medeni Durum'].replace({'Bekar': 0, 'Evli': 1})

df_train['Eğitime Devam Etme Durumu'] = df_train['Eğitime Devam Etme Durumu'].replace({'Etmiyor': 0, 'Ediyor': 1})
df_test['Eğitime Devam Etme Durumu']  = df_test['Eğitime Devam Etme Durumu'].replace({'Etmiyor': 0, 'Ediyor': 1})

In [13]:
kategorik_kolonlar = ['Yaş Grubu','Eğitim Düzeyi','Yaşadığı Şehir', 'İstihdam Durumu', 'En Çok İlgilendiği Ürün Grubu']
encoded_train = pd.get_dummies(df_train, columns=kategorik_kolonlar)
encoded_test = pd.get_dummies(df_test, columns=kategorik_kolonlar)

X = encoded_train.drop('Öbek İsmi', axis=1)
y = encoded_train['Öbek İsmi']

# MODEL

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Regülarizasyon parametrelerini belirleyin
params = {
    'n_estimators': 100,
    'random_state': 42,
    'colsample_bytree': 0.8,  # Özellik seçimi
    'reg_alpha': 0.1,  # L1 düzenlemesi = Bunun 0 veya çok düşük
    'reg_lambda': 1   # L2 düzenlemesi = Bunun yüksek olması gerektiğini söylüyorlar
}

model = LGBMClassifier(**params)
model.fit(X_train, y_train)

scores = cross_val_score(model, X_train, y_train, cv=5) 
print("Cross-Validation Scores:", scores)
print("Mean CV Score:", scores.mean())

y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Doğruluk
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Hassasiyet
precision = precision_score(y_test, y_pred, average='macro')
print("Precision:", precision)

# Duyarlılık
recall = recall_score(y_test, y_pred, average='macro')
print("Recall:", recall)

# F1-Skoru
f1 = f1_score(y_test, y_pred, average='macro')
print("F1-Score:", f1)

Cross-Validation Scores: [0.96462264 0.97287736 0.95754717 0.97169811 0.95990566]
Mean CV Score: 0.9653301886792451
Confusion Matrix:
[[135   0   0   2   2   0   1   1]
 [  1 107   0   0   0   0   1   0]
 [  0   0 137   1   0   1   0   0]
 [  0   0   0 118   3   0   2   3]
 [  1   0   1   0 141   0   0   1]
 [  0   0   2   0   0 137   2   0]
 [  1   0   2   0   1   0 123   0]
 [  0   0   0   0   0   0   0 133]]
Accuracy: 0.9726415094339622
Precision: 0.9734312497778347
Recall: 0.9725649301631287
F1-Score: 0.972834445591735


In [15]:
from sklearn.metrics import classification_report

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

      obek_1       0.98      0.96      0.97       141
      obek_2       1.00      0.98      0.99       109
      obek_3       0.96      0.99      0.98       139
      obek_4       0.98      0.94      0.96       126
      obek_5       0.96      0.98      0.97       144
      obek_6       0.99      0.97      0.98       141
      obek_7       0.95      0.97      0.96       127
      obek_8       0.96      1.00      0.98       133

    accuracy                           0.97      1060
   macro avg       0.97      0.97      0.97      1060
weighted avg       0.97      0.97      0.97      1060



In [16]:
X = encoded_train.drop('Öbek İsmi', axis=1)
y = encoded_train['Öbek İsmi']

model = LGBMClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

submission_pred = model.predict(encoded_test)

submission_df = pd.DataFrame({'id': encoded_test.index, 'obek_ismi': submission_pred})

submission_df.to_csv('submission.csv', index=False)

In [17]:
submission_pred = model.predict(encoded_test)

submission_df = pd.DataFrame({'id': encoded_test.index, 'Öbek İsmi': submission_pred})

submission_df.set_index('id', inplace=True)

submission_df.to_csv('submission.csv')

In [18]:
submission_df

,Öbek İsmi
id,
0,obek_3
1,obek_3
2,obek_2
3,obek_6
4,obek_1
...,...
2335,obek_8
2336,obek_2
2337,obek_7
